In [ ]:
from MCTS import MT
from transformers import AutoTokenizer, AutoModelForCausalLM
from comet import download_model, load_from_checkpoint
import torch
import os

In [ ]:
# initialization variable
INPUT_PATH = 'src/zh-en'
OUTPUT_PATH = 'output/results'
# the name/path of the translation model
LLM_PATH = 'Unbabel/TowerInstruct-7B-v0.2'
# the evaluation model name(must be reference-free)
# recommend cometkiwi, other options may need code adjustments.
EVA_PATH = 'Unbabel/wmt22-cometkiwi-da'
# the gpu number
gpu_map = '6,7'
# the score threshold that determine if the translation pair will be logged
threshold = [0.04,0.4]
# correspond with the translation direction
# e.g. from Chinese to English
language_pair = ['Chinese', 'English']

In [ ]:
# load the models
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_map
llm = AutoModelForCausalLM.from_pretrained(LLM_PATH, torch_dtype=torch.bfloat16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(LLM_PATH)

model_path = download_model(EVA_PATH)
eva = load_from_checkpoint(model_path)
# Or you can load from your local device
# eva = load_from_checkpoint(EVA_PATH)

In [ ]:
mt = MT(llm, tokenizer, eva, threshold, OUTPUT_PATH, language_pair)

In [ ]:
# read the source sentence
f = open(INPUT_PATH, 'r', encoding='utf-8')
lines = f.read().splitlines()
f.close()

for line in lines:
    mt.process(line)